# Análisis de índices bursátiles

## Importación de librerías

In [107]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.dates as mdates
import plotly.express as px
import os


## Carga de datos

In [108]:
df_sp500 = pd.read_csv("../data/raw/sp_500.csv")
df_cac40 = pd.read_csv("../data/raw/cac_40.csv")
df_dax40 = pd.read_csv("../data/raw/dax_40.csv")
df_dowjones = pd.read_csv("../data/raw/dow_jones.csv")
df_eurostoxx50 = pd.read_csv("../data/raw/euro_stoxx_50.csv")
df_ftse100 = pd.read_csv("../data/raw/ftse_100.csv")
df_hangseng = pd.read_csv("../data/raw/hang_seng.csv")
df_nasdaq100 = pd.read_csv("../data/raw/nasdaq_100.csv")
df_nikkei225 = pd.read_csv("../data/raw/nikkei_225.csv")
df_russell2000 = pd.read_csv("../data/raw/russell_2000.csv")


## Procesamiento y Limpieza de datos

📈 Función de análisis financiero por índice:  

Esta función aplica un conjunto de cálculos estándar de análisis financiero a cualquier índice bursátil. Dado un DataFrame con precios diarios (Open, High, Low, Close, Volume), añade columnas clave para evaluar su rendimiento y comportamiento a lo largo del tiempo.

🔧 Cálculos incluidos:  
- Rentabilidad diaria y porcentual
- Medias móviles de 10 y 50 días
- Rentabilidad acumulada (desde el primer día)
- Caída máxima (drawdown) respecto al máximo histórico
- Volatilidad móvil de 30 días
- Ratio de Sharpe móvil de 30 días

📊 Indicadores agregados que se imprimen:
- CAGR (Tasa de crecimiento anual compuesta)
- Caída máxima promedio
- Volatilidad media
- Sharpe medio

In [109]:
def procesar_indice(df, nombre='Índice'):
    df['rentabilidad'] = df['close'].pct_change()
    df['rentabilidad_%'] = df['rentabilidad'] * 100
    df['media_movil_10d'] = df['close'].rolling(window=10).mean()
    df['media_movil_50d'] = df['close'].rolling(window=50).mean()
    df['rentabilidad_acumulada'] = (1 + df['rentabilidad']).cumprod()
    df['maximo_acumulado'] = df['rentabilidad_acumulada'].cummax()
    df['caida_maxima'] = df['rentabilidad_acumulada'] / df['maximo_acumulado'] - 1
    df = df.drop(columns=['maximo_acumulado'])
    df['volatilidad_30d'] = df['rentabilidad'].rolling(window=30).std()
    df['sharpe_30d'] = df['rentabilidad'].rolling(window=30).mean() / df['volatilidad_30d']

    años = (pd.to_datetime(df.index[-1]) - pd.to_datetime(df.index[0])).days / 365
    precio_inicial = df['close'].iloc[0]
    precio_final = df['close'].iloc[-1]

    indicadores = {
        'CAGR': (precio_final / precio_inicial) ** (1 / años) - 1,
        'drawdown_promedio': df['caida_maxima'].mean(),
        'volatilidad_media': df['volatilidad_30d'].mean(),
        'sharpe_medio': df['sharpe_30d'].mean()
    }

    return df, indicadores



In [110]:

nombres_dfs = [
    'df_sp500', 'df_cac40', 'df_dax40', 'df_dowjones', 'df_eurostoxx50',
    'df_ftse100', 'df_hangseng', 'df_nasdaq100', 'df_nikkei225', 'df_russell2000'
]

for nombre in nombres_dfs:
    globals()[nombre] = globals()[nombre][['Date', 'Open', 'High', 'Low', 'Close', 'Volume']]
    globals()[nombre].columns = globals()[nombre].columns.str.lower()

nombres_dfs = [
    "df_sp500",
    "df_cac40",
    "df_dax40",
    "df_dowjones",
    "df_eurostoxx50",
    "df_ftse100",
    "df_hangseng",
    "df_nasdaq100",
    "df_nikkei225",
    "df_russell2000"
]

for nombre in nombres_dfs:
    globals()[nombre].set_index("date", inplace=True)




Crear nueva tabla comparativa entre indices

In [114]:
# Lista para guardar los resultados de cada índice
all_indices = []

ruta_guardado = '../data/processed'

# Diccionario con los nombres y los DataFrames
indices = {
    "S&P 500": df_sp500,
    "CAC 40": df_cac40,
    "DAX 40": df_dax40,
    "Dow Jones": df_dowjones,
    "Euro Stoxx 50": df_eurostoxx50,
    "FTSE 100": df_ftse100,
    "Hang Seng Index": df_hangseng,
    "Nasdaq 100": df_nasdaq100,
    "Nikkei 225": df_nikkei225,
    "Russell 2000": df_russell2000
}

# Bucle principal
for nombre, df in indices.items():
    df_procesado, resumen = procesar_indice(df.copy(), nombre)

    # Añadir los resultados al resumen general
    all_indices.append({
        "indice": nombre,
        "crec_anu_comp": resumen["CAGR"],
        "caida_maxima_promedio": resumen["drawdown_promedio"],
        "volatilidad_media": resumen["volatilidad_media"],
        "sharpe_medio": resumen["sharpe_medio"]
    })

# Crear DataFrame resumen
df_all_indices = pd.DataFrame(all_indices)

# Guardar el resumen en la ruta deseada
ruta_resumen = os.path.join(ruta_guardado, 'all_indices.csv')
df_all_indices.to_csv(ruta_resumen, index=False)
print(f"Resumen guardado en {ruta_resumen}")


Resumen guardado en ../data/processed/all_indices.csv


In [ ]:
# Diccionario con tus DataFrames
indices = {
    "S&P 500": df_sp500,
    "CAC 40": df_cac40,
    "DAX 40": df_dax40,
    "Dow Jones": df_dowjones,
    "Euro Stoxx 50": df_eurostoxx50,
    "FTSE 100": df_ftse100,
    "Hang Seng Index": df_hangseng,
    "Nasdaq 100": df_nasdaq100,
    "Nikkei 225": df_nikkei225,
    "Russell 2000": df_russell2000
}

def nombre_a_archivo(nombre):
    nombre = nombre.lower()
    nombre = nombre.replace(" ", "_")
    nombre = nombre.replace("&", "")
    nombre = nombre.replace(".", "")
    nombre = nombre.replace("-", "_")
    return nombre + '.csv'

for nombre, df in indices.items():
    df_procesado, _ = procesar_indice(df.copy(), nombre)  # Desempaquetar la tupla
    archivo = nombre_a_archivo(nombre)
    ruta = os.path.join(ruta_guardado, archivo)
    df_procesado.to_csv(ruta)
    print(f"Guardado {nombre} en {ruta}")



Guardado S&P 500 en ../data/processed/sp_500.csv
Guardado CAC 40 en ../data/processed/cac_40.csv
Guardado DAX 40 en ../data/processed/dax_40.csv
Guardado Dow Jones en ../data/processed/dow_jones.csv
Guardado Euro Stoxx 50 en ../data/processed/euro_stoxx_50.csv
Guardado FTSE 100 en ../data/processed/ftse_100.csv
Guardado Hang Seng Index en ../data/processed/hang_seng_index.csv
Guardado Nasdaq 100 en ../data/processed/nasdaq_100.csv
Guardado Nikkei 225 en ../data/processed/nikkei_225.csv
Guardado Russell 2000 en ../data/processed/russell_2000.csv
